In [2]:
import chess
import chess.engine

# Create a new chess board
board = chess.Board()

# Print the board
print(board)

# Check if the game is over
def is_game_over(board):
    return board.is_checkmate() or board.is_stalemate() or board.is_insufficient_material()

# Function to play a move
def play_move(board, move):
    board.push(move)

# Example random move for white
# print("White's Move:")
# move = chess.Move.from_uci('e2e4')
# play_move(board, move)
# print(board)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


In [3]:
import math

# Evaluate the board (basic material evaluation)
def evaluate_board(board):
    if board.is_checkmate():
        if board.turn:
            return -9999  # White is in checkmate (black wins)
        else:
            return 9999   # Black is in checkmate (white wins)
    elif board.is_stalemate() or board.is_insufficient_material():
        return 0  # Draw
    else:
        # Material evaluation (simplified: queens = 9, rooks = 5, bishops/knights = 3, pawns = 1)
        material_scores = {chess.PAWN: 1, chess.KNIGHT: 3, chess.BISHOP: 3, chess.ROOK: 5, chess.QUEEN: 9}
        eval_score = 0
        for piece_type in material_scores:
            eval_score += len(board.pieces(piece_type, chess.WHITE)) * material_scores[piece_type]
            eval_score -= len(board.pieces(piece_type, chess.BLACK)) * material_scores[piece_type]
        return eval_score

# Minimax with Alpha-Beta Pruning
def minimax(board, depth, alpha, beta, maximizing_player):
    if depth == 0 or board.is_game_over():
        return evaluate_board(board)
    
    if maximizing_player:
        max_eval = -math.inf
        for move in board.legal_moves:
            board.push(move)
            eval = minimax(board, depth - 1, alpha, beta, False)
            board.pop()
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = math.inf
        for move in board.legal_moves:
            board.push(move)
            eval = minimax(board, depth - 1, alpha, beta, True)
            board.pop()
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

In [4]:
# Choose the best move for the AI using Minimax
def best_move(board, depth):
    best_eval = -math.inf
    best_move = None
    for move in board.legal_moves:
        board.push(move)
        move_eval = minimax(board, depth - 1, -math.inf, math.inf, False)
        board.pop()
        if move_eval > best_eval:
            best_eval = move_eval
            best_move = move
    return best_move

In [5]:
def play_game():
    board = chess.Board()

    while not is_game_over(board):
        print(board)

        # Player's move (e.g., you can use input() to get UCI input)
        player_move = input("Enter your move (UCI notation): ")
        try:
            move = chess.Move.from_uci(player_move)
            if move in board.legal_moves:
                board.push(move)
            else:
                print("Illegal move! Try again.")
        except:
            print("Invalid move format! Use UCI format.")

        if is_game_over(board):
            break

        # AI's move
        print("AI is thinking...")
        ai_move = best_move(board, 3)  # You can adjust the depth of search
        board.push(ai_move)
        print(f"AI played: {ai_move}")

        if is_game_over(board):
            break

    print("Game Over")
    print(board)

In [17]:
import pygame
import chess
import chess.engine

# Initialize Pygame
pygame.init()

# Set up the screen
screen = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
pygame.display.set_caption("Chess Game")

# Load piece images
piece_images = {
    'P': pygame.image.load(r"C:\Users\DELL\Dropbox\My PC (DESKTOP-RU4OP8A)\Desktop\chess\pawnChess_plt45.svg.png"),
    'R': pygame.image.load(r"C:\Users\DELL\Dropbox\My PC (DESKTOP-RU4OP8A)\Desktop\chess\rookChess_tile_rl-whitebg.svg.png"),
    'N': pygame.image.load(r"C:\Users\DELL\Dropbox\My PC (DESKTOP-RU4OP8A)\Desktop\chess\knightChess_nlt45.svg.png"),
    'B': pygame.image.load(r"C:\Users\DELL\Dropbox\My PC (DESKTOP-RU4OP8A)\Desktop\chess\bishopChess_blt45.svg.png"),
    'Q': pygame.image.load(r"C:\Users\DELL\Dropbox\My PC (DESKTOP-RU4OP8A)\Desktop\chess\Chess_qlt45queen.svg.png"),
    'K': pygame.image.load(r"C:\Users\DELL\Dropbox\My PC (DESKTOP-RU4OP8A)\Desktop\chess\Chess_klt45.svg.png"),
    'p': pygame.image.load(r"C:\Users\DELL\Dropbox\My PC (DESKTOP-RU4OP8A)\Desktop\chess\pawnChess_plt45.svg.png"),
    'r': pygame.image.load(r"C:\Users\DELL\Dropbox\My PC (DESKTOP-RU4OP8A)\Desktop\chess\rookChess_tile_rl-whitebg.svg.png"),
    'n': pygame.image.load(r"C:\Users\DELL\Dropbox\My PC (DESKTOP-RU4OP8A)\Desktop\chess\knightChess_nlt45.svg.png"),
    'b': pygame.image.load(r"C:\Users\DELL\Dropbox\My PC (DESKTOP-RU4OP8A)\Desktop\chess\bishopChess_blt45.svg.png"),
    'q': pygame.image.load(r"C:\Users\DELL\Dropbox\My PC (DESKTOP-RU4OP8A)\Desktop\chess\Chess_qlt45queen.svg.png"),
    'k': pygame.image.load(r"C:\Users\DELL\Dropbox\My PC (DESKTOP-RU4OP8A)\Desktop\chess\Chess_klt45.svg.png"),
}

def draw_board(board):
    colors = [pygame.Color("white"), pygame.Color("gray")]
    square_size = screen.get_width() // 8  # Dynamically set square size based on screen width
    for row in range(8):
        for col in range(8):
            color = colors[(row + col) % 2]
            pygame.draw.rect(screen, color, pygame.Rect(col * square_size, row * square_size, square_size, square_size))

            piece = board.piece_at(row * 8 + col)
            if piece is not None:
                # Scale images to fit the squares
                piece_image = piece_images[piece.symbol()]
                piece_image = pygame.transform.scale(piece_image, (square_size, square_size))
                screen.blit(piece_image, (col * square_size, row * square_size))

# Main game loop
running = True
board = chess.Board()

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill(pygame.Color("black"))
    draw_board(board)
    pygame.display.flip()

pygame.quit()